In [18]:
# Carga de paquetes
import pandas_datareader as pdr
from pandas_datareader import data, wb
from datetime import date
import yfinance as yf
import numpy as np
import pandas as pd

In [22]:
# Definir un retorno para el activo libre de riesgo
risk_free_return = 0.05

# Definir fechas
start = pd.to_datetime('2010-01-01')
end = pd.to_datetime('2021-01-01')

# Definir las acciones
ticker_1 = 'FERREYC1.LM' # Accion de Ferreyros
ticker_2 = '^SPBLPGPT'# Indice de la BVL


def capm(start, end, ticker1, ticker2):
    # Obtenemos la data de la acción
    stock_1 = yf.download(ticker_1, start=start, end=end)['Adj Close']
    # Obtenemos la data del mercado
    stock_2 = yf.download(ticker_2, start=start, end=end)['Adj Close']

    # Reasampleado para data mensual
    return_s1 = stock_1.resample('M').last()
    return_s2 = stock_2.resample('M').last()

    # Creamos un dataframe con los precios de cierre
    data = pd.DataFrame({'Inv_Close': return_s1, 'Markt_Close': return_s2})

    # Calcular los retornos logarítmicos de la acción e índice
    data[['Inv_Ret', 'Markt_Ret']] = np.log(data[['Inv_Close', 'Markt_Close']] / data[['Inv_Close', 'Markt_Close']].shift(1))

    # Eliminar los valores nulos generados por el shift
    data.dropna(inplace=True)

    # Calcular la beta usando la fórmula del CAPM
    beta_form = (data[['Inv_Ret', 'Markt_Ret']].cov().iloc[0, 1]) / data['Markt_Ret'].var()

    # Mostrar el resultado de la beta
    print('Beta de la fórmula del CAPM: ', beta_form.round(4))

    # Calculamos el retorno esperado
    expected_return = risk_free_return + beta*(data["m_returns"].mean()*12-risk_free_return)
    print("Retorno esperado: ",expected_return)

if __name__ == "__main__":
    capm("2010-01-01", "2021-01-01", 'S', "^GSPC")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


KeyError: 's_returns'